まずは単語の入ったcsvファイルを読み込む。項目数が多いので、DictReaderで辞書として取り込み、それをリスト化する

In [31]:
import csv
word_list = []
with open('chinesewords_20160219.csv', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        word_list.append(row)
print('{} words stored in word_list'.format(len(word_list)))

5809 words stored in word_list


同じく単文の入ったtxtファイルを読み込む。こちらは項目数が今のところ1つだけなので、csvとして読む必要はない。ただし、行末の改行(\n)を消してやる必要あり。

In [51]:
sentence_list = []
with open('chinese_sentences20160218.txt', encoding='utf-8') as textfile:
    sentence_list = [row.rstrip() for row in textfile]
print('{} sentences stored in sentenc_list'.format(len(sentence_list)))

569 sentences stored in sentenc_list


改行が消えているかの確認。

In [52]:
sentence_list[-1]

'那么大的鱼，根本钓不到。'

In [53]:
for sentence in sentence_list:
    print(sentence)

再挨紧一点儿，要不然照不进去。
挨近一看，原来是一片树叶。
他们两个挨得很近，她是他的女朋友吧？
他爱上了一个同班同学。
爱不起来也不能强迫。
他们的房间都安排好了。
安排完明天的工作，他才回家。
这个计划安排的很不科学。
旅行社给我们安排得很周到。
他的生活安排得一塌糊涂。
招待外国朋友，主人有时候也会把卡拉OK安排到日程里去。
语法课就安排在星期一吧。
他迅速地拿出零件安装的上去。
什么时候能安装好啊？
那位老师傅安装得快极了。
你能把这个安装到原来的地方去吗？
糟糕！我按错了地方，把刚写的文章消掉了。
请按住，别松手！
没带图章的话，在这儿按一个手印也行，就按在这儿吧。
把那个钉子拔下来吧。
昨天他去把虫牙拔掉了。
他把点心掰成一小块一小块的，往孩子嘴里送。
他摆出一副长辈的样子。
你可以把这样做的理由都摆出来。
把公筷也摆上吧。
我的屋里摆满各种观赏植物。
要摆平关系还真不容易。
各种新商品在柜台里摆得整整齐齐的。
这张桌子很大，所有的资料都摆得下。
看着摆在盘子里的焦黄的油炸虾，我情不自禁地高兴起来。
要讨论的问题都到摆桌面上来吧。
你就决心自己干吧，谁也拜托不了。
要打扫卫生，教室里的桌椅都得搬出去。
你搬好家了吗？
住户搬走了，押金退给他了吗？
们这么小，这个长沙发搬得进去搬不进去？
他刚搬到我隔壁来。
这十年中，我搬了两次家。
那家公司条件很好，他正在想办法把小王办进去。
这个案子要办下去，一定要把案情及有关人员弄清楚。
办好入境手续我就放心了。
她很能干，事情办得很漂亮。
这件事很复杂，不知你办得了办不了？
请你给我包上。
我包得不好，他包得很好，你找他去吧。
这几天天气特别冷，出去的时候她把孩子包得严严实实的。
春节去饭店吃饭包不到单间。
这个东西真不好包，怎么包也包不紧。
这件事包在我身上了。
歌星一走出来，等候在出口的歌迷们就立刻把他包围了起来。
四周的山把这山谷包围得像一口井。
她的记录恐怕要保持不了了。
看这个吉尼斯纪录能保持到什么时候？
这种习惯一直保持了很久。
这是纪念邮票，保存起来。
没想到妻子会把丈夫的情书保存到现在。
他把保存了多年的旧胶片捐赠给音像图书馆了。
快把现场保护起来。
我保护不了他。
他的学籍会一直保留下去的。
您的座位我可以给您保留到明天。
保留一年两年大概没问题。
他把小妹妹抱了起来。
你把小核儿抱上楼去吧。
这

Neo4jとつないでみよう。あらかじめpy2neoをインストールしておく。
```
$pip install py2neo
```

In [64]:
from py2neo import Graph, authenticate
from py2neo import Node, Relationship
authenticate('localhost:7474', 'neo4j', 'admin')
graph = Graph()    #http://localhost:7474

In [77]:
for sentence in sentence_list:
    new_sentence = Node("Sentence", sentence=sentence)
    graph.create(new_sentence)

単語と品詞(Category)をNodeとして追加、さらにそれらのrelationshipを登録する。Categoryを重複登録させたくないので、工夫が必要。

In [79]:
counter_c = 0
counter_w = 0
counter_a = 0
for word in word_list:
    new_word = Node("Word", keyword=word['word'], pinyin=word['pinyin'], pinyinf=word['pinyinf'], meaning=word['meaning'])
    if word.get('category'):
        counter_c += 1
        counter_a += 1
        new_category = Node("Category", category=word['category'])
        word_is_categorized_as_category = Relationship(new_word, 'is_categorized_as', new_category)
        graph.merge(word_is_categorized_as_category)
    else:
        counter_w += 1
        counter_a += 1
        graph.merge(new_word)
print("all count:{0}, with_category:{1}, only_words:{2}".format(counter_a, counter_c, counter_w))    

all count:5809, with_category:1209, only_words:4600
